<a href="https://colab.research.google.com/github/CAWaterBoardDataCenter/OpenET_Queries/blob/main/Public_Queries.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# access token
api_key = "" #Add your key within the ""

In [ ]:
import json
import requests

# remove warning message
requests.packages.urllib3.disable_warnings(requests.packages.urllib3.exceptions.InsecureRequestWarning)

def call_api(endpoint, api_key, args, post=True):
    """Using user specified inputs, returns data from OpenET's API.

    Args:
        endpoint (str): Raster API endpoint

        api_key (str): Required api access key

        args (dictionary): User specified arguments for api call

    Returns:
        result (object): An object of Raster API results
    """

    # api server address
    server = 'https://openet.dri.edu/'

    # initialize request url
    url = server + endpoint

    # create header
    header = {"Authorization": api_key}

    # make api request
    if post == False:
        resp = requests.get(url=url, headers=header, params=args, verify=False)
    else:
        resp = requests.post(url=url, data=json.dumps(args), headers=header, verify=False)

    # view results
    return resp.text
    

In [ ]:
# raster api endpoint
endpoint = 'raster/timeseries/multipolygon'

# endpoint arguments
args = {
    # temporal options
    'start_date' : '2022-8-01',                                  # inclusive starting date
    'end_date'   : '2022-10-01',                                  # inclusive completion date
    'interval'  : 'monthly',
    # spatial options
    'shapefile_asset_id' : 'users/lindsaykammeier/DDRPP_Merged_220927_1_30m_Elev',    #UPDATE file name from Asset > Table ID https://code.earthengine.google.com/ (Don't forget to share file with openet@googlegroups.com) 
    'include_columns'   : 'Name,DDRPP_Nm,Sort_Typ,Proj_Adj,2021_Crop,2022_Crop,_mean,_min,_max',                                  #UPDATE to column names specific for file [DDRPP_Nm,Sort_Typ,Proj_Adj,Crop_2022,Crop_2021,descrip,layer]
    # OpenET options
    'variable'      : 'et',                                       # variable to retrieve (ndvi, etof, eto, et, pr)
    'model'         : 'ensemble',                                 # model selection (ensemble, geesebal, ssebop, eemetric, sims, disalexi, ptjpl)
    'ref_et_source' : 'cimis',                                    # reference et collection (cimis, gridmet)
    'provisional'  : 'true',
    # data processing options
    'units'         : 'english',                                  # output units (metric [mm], english [in])
    'filename_suffix' : 'public_export'                             # filename to append
}

vars = ['et','et_mad_min','et_mad_max']                                      #Options: et, eto, pr; ndvi?
models = ['ensemble']                           #What are options? ensemble, range, eemetric, ssebop, sims, pt-jpl, disalexi, geesebal
for var in vars:
    for model in models:

        # update
        args.update({'variable': var})
        args.update({'model': model})
    
        # query result
        response = call_api(endpoint, api_key, args, post=False)

        # cast as a dictionary 
        response = json.loads(response)
        print(response)